In [4]:
import pandas as pd
import numpy as np
import re
import os

# Data cleaning and preprocessing

# Nuforc data

In [8]:
# Nuforc data comes from Kaggle: https://www.kaggle.com/NUFORC/ufo-sightings
data = pd.read_csv('../data/raw/scrubbed.csv', low_memory=False, on_bad_lines= 'skip') # low_memory=False to avoid mixed types warning
data.head()

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


# Data cleaning and preprocessing

## Steps to clean the data

1. Check for missing values: Identify and handle missing values in the data, either by dropping rows with missing values or by imputing the missing values with suitable replacements.

2. Check for inconsistencies: Look for inconsistencies in the data, such as typos or misclassified categories, and correct them as needed.

3. Remove duplicates: Remove duplicate rows from the data to avoid double-counting reports.

4. Normalize text data: Normalize the text data by lowercasing all letters, removing punctuation, and stemming or lemmatizing the words to reduce the dimensionality of the data.

5. Check for invalid or outlier values: Check for invalid or outlier values in the data and handle them appropriately. For example, you may want to remove or replace extreme values that could affect your analysis.

6. Standardize date and time formats: Make sure that the date and time formats are consistent across all records.



In [9]:
# step 1 - drop columns with too many missing values
df = data.copy()
#& Function to print the number of missing values in each column
def print_missing(df):
    # Count the number of missing values in each column
    missing = df.isnull().sum()
    
    # Iterate over the columns and print a statement for each column
    for col in df.columns:
        num_missing = missing[col]
        if num_missing > 0:
            # calc the percentage of missing values versus total values in the column
            percent_missing = num_missing / df.shape[0] * 100
            print(f"The '{col}' column has {num_missing} missing values, which is {percent_missing}% of the total values.")
            # show the value counts of the column if it's a categorical column and there are missing values in it.
            if df[col].dtype == 'object': # if the column is a categorical column
                print(f"Value counts for the '{col}' column: {df[col].value_counts()}")
        else:
            print(f"The '{col}' column has no missing values.")

# print the number of missing values in each column
print_missing(df)

The 'datetime' column has no missing values.
The 'city' column has no missing values.
The 'state' column has 5797 missing values, which is 7.216302345267141% of the total values.
Value counts for the 'state' column: ca    9655
wa    4268
fl    4200
tx    3677
ny    3219
      ... 
nf      25
nt      20
pe      17
yt      13
yk       7
Name: state, Length: 67, dtype: int64
The 'country' column has 9670 missing values, which is 12.037544191604841% of the total values.
Value counts for the 'country' column: us    65114
ca     3000
gb     1905
au      538
de      105
Name: country, dtype: int64
The 'shape' column has 1932 missing values, which is 2.405019170442663% of the total values.
Value counts for the 'shape' column: light        16565
triangle      7865
circle        7608
fireball      6208
other         5649
unknown       5584
sphere        5387
disk          5213
oval          3733
formation     2457
cigar         2057
changing      1962
flash         1328
rectangle     1297
cylind

I am going to drop the state and country columns, but I might want to consider keeping the city column. It could be useful for analysis at the city level, particularly if we are interested in comparing UFO sightings across different cities. Additionally, depending on the level of detail there is in the latitude and longitude columns, the city column might provide valuable context for understanding the location of the UFO sighting.

In [10]:
# drop columns: country, state
df_original = df.copy() # save a copy of the original dataframe
df = df_original.drop(columns=['country', 'state'])

In [11]:
#print_missing(df)
# duration (seconds) has 2 missing values - drop these rows
df = df.dropna(subset=['duration (seconds)'])
# drop the 35 rows with missing values in the 'comments' column
df = df.dropna(subset=['comments'])
df.isnull().sum()

datetime                   0
city                       0
shape                   1925
duration (seconds)         0
duration (hours/min)       0
comments                   0
date posted                0
latitude                   0
longitude                  0
dtype: int64

Shapes could be mentioned in the comments so I am not going to drop based on missing shapes.

Also, if there is a duration in seconds but not in hours we could use seconds as a proxy for the missing values in the hours column

In [12]:
# get the length of each string in the 'comments' column where the 'shape' column is null
comments_lengths = df[df['shape'].isnull()]['comments'].str.len()
# drop the rows where the 'shape' column is null and comments length is less than 10 characters long as these are not as likely to include shape descriptions (e.g. "I saw a light in the sky") vs longer comments (e.g. "I saw a light in the sky that was shaped like a triangle")
df = df.drop(df[(df['shape'].isnull()) & (df['comments'].str.len() < 10)].index)
# view the distribution of the lengths
comments_lengths.describe()


count    1925.000000
mean      105.230649
std        41.624632
min         3.000000
25%        71.000000
50%       122.000000
75%       138.000000
max       169.000000
Name: comments, dtype: float64

In [13]:
df.isnull().sum()


datetime                   0
city                       0
shape                   1917
duration (seconds)         0
duration (hours/min)       0
comments                   0
date posted                0
latitude                   0
longitude                  0
dtype: int64

In [14]:
import re

# get the unique shapes in the 'shapes' column
unique_shapes = df['shape'].unique() # array of unique shapes
unique_shapes = [str(shape).lower() for shape in unique_shapes] # convert to lowercase
# remove any null values
unique_shapes = [shape for shape in unique_shapes if str(shape) != 'nan']
# remove any empty strings
unique_shapes = [shape for shape in unique_shapes if shape != '']
# remove any duplicates
unique_shapes = list(set(unique_shapes))
# sort the list
unique_shapes.sort()
# print the unique shapes
print(unique_shapes)

# create a regex pattern that matches any of the unique shapes
regex_pattern = '|'.join(unique_shapes)

# test the regex pattern
test_string = "I saw a light in the sky that was shaped like a triangle"
match = re.findall(regex_pattern, test_string)
if match:
    print(match)
else:
    print("No match")


['changed', 'changing', 'chevron', 'cigar', 'circle', 'cone', 'crescent', 'cross', 'cylinder', 'delta', 'diamond', 'disk', 'dome', 'egg', 'fireball', 'flare', 'flash', 'formation', 'hexagon', 'light', 'other', 'oval', 'pyramid', 'rectangle', 'round', 'sphere', 'teardrop', 'triangle', 'unknown']
['light', 'triangle']


In [15]:
df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [16]:
def extract_shapes(comments):
    # use the regex pattern to extract all occurrences of the shapes in the comments
    shapes = re.findall(regex_pattern, comments)
    # return a string of the shapes separated by a space
    return ' '.join(shapes)

# create a new column called 'comments_shapes' that contains the extracted shapes
df['comments_shapes'] = df['comments'].apply(extract_shapes)

In [17]:
df['duration (hours/min)'].value_counts()[0:40]

5 minutes       4715
2 minutes       3500
10 minutes      3322
1 minute        3060
3 minutes       2517
30 seconds      2323
15 minutes      2067
10 seconds      1992
5 seconds       1813
20 minutes      1454
30 minutes      1330
1 hour          1328
15 seconds      1175
20 seconds      1133
3 seconds        933
4 minutes        904
5 min            846
2 seconds        691
2 hours          652
10 min           645
2-3 minutes      613
2 min            540
45 seconds       524
seconds          518
1-2 minutes      498
45 minutes       495
1 min            488
15 min           469
4 seconds        448
3 min            413
5-10 minutes     368
5 mins           361
5 min.           344
20 min           341
3-4 minutes      333
1 second         332
7 minutes        329
3-5 minutes      325
60 seconds       312
30 min           297
Name: duration (hours/min), dtype: int64

In [18]:
df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comments_shapes
0,10/10/1949 20:30,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,round
1,10/10/1949 21:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,cross
2,10/10/1955 17:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,
3,10/10/1956 21:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,other
4,10/10/1960 20:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,


# Time Cleaning

In [19]:
import re

# define a dictionary where the keys are the time units and the values are the possible ways they may be represented
time_units = {
    'minutes': ['minutes', 'minute', 'mins', 'min.', 'min'],
    'seconds': ['seconds', 'second', 'secs', 'sec'],
    'hours': ['hours', 'hour', 'hrs', 'hr'],
    'days': ['days', 'day', 'd'],
    'weeks': ['weeks', 'week', 'wks', 'wk'],
    'months': ['months', 'month'],
    'years': ['years', 'year', 'yrs'],
    'am': ['a.m.', 'a.m', 'am'],
    'pm': ['p.m.', 'p.m', 'pm'],
    'midnight': ['midnight', 'midnite', 'mid nite', 'mid-nite'],
    'unknown': ['unknown', 'unk']
}



# create a list of regex patterns for each time unit
patterns = []
for time_unit, representations in time_units.items():
    # create a regex pattern for each representation of the time unit
    for representation in representations:
        pattern = rf"(\d+)\s*{representation}"
        patterns.append(pattern)

# join the patterns into a single regex pattern
regex_pattern = '|'.join(patterns)

# test the regex pattern
test_string = "I saw a light in the sky that was there for 5 minutes"
match = re.search(regex_pattern, test_string)
if match:
    print(match.group())
else:
    print("No match")


5 minutes


In [20]:
df['calculated_duration'] = df['duration (hours/min)'].apply(
    lambda x: re.search(regex_pattern, x).group() if pd.notnull(x) and re.search(regex_pattern, x) else x)
df['duration_value'] = df['calculated_duration'].apply(lambda x: int(re.search(r'\d+', x).group()) if pd.notnull(x) and re.search(r'\d+', x) else x)
df['duration_unit'] = df['calculated_duration'].apply(lambda x: re.search(regex_pattern, x).group().replace(str(re.search(r'\d+', x).group()), '').strip() if pd.notnull(x) and re.search(regex_pattern, x) else x)



In [21]:
df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comments_shapes,calculated_duration,duration_value,duration_unit
0,10/10/1949 20:30,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,round,45 minutes,45,minutes
1,10/10/1949 21:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,cross,2 hrs,2,hrs
2,10/10/1955 17:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,,20 seconds,20,seconds
3,10/10/1956 21:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,other,2 hour,2,hour
4,10/10/1960 20:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,,15 minutes,15,minutes


In [22]:
# define a function that takes in a string and returns the corresponding key in the time_units dictionary
def replace_time_unit(string):
    # iterate over the keys and values in the time_units dictionary
    for key, values in time_units.items():
        # check if the string is in the list of values
        if string in values:
            # if it is, return the key
            return key

# use the apply method to apply the replace_time_unit function to the duration_unit column
df['duration_unit'] = df['duration_unit'].apply(replace_time_unit)
df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comments_shapes,calculated_duration,duration_value,duration_unit
0,10/10/1949 20:30,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,round,45 minutes,45,minutes
1,10/10/1949 21:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,cross,2 hrs,2,hours
2,10/10/1955 17:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,,20 seconds,20,seconds
3,10/10/1956 21:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,other,2 hour,2,hours
4,10/10/1960 20:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,,15 minutes,15,minutes


In [23]:
#Finally, create a column `calculated_duration_value` and convert the duration_value number into seconds based on the value in the `duration_unit` column (i.e., 45 and minutes means 45 * 60 seconds).
df['duration_value'] = pd.to_numeric(df['duration_value'], errors='coerce')
# Set default value for time units that are not in the dictionary
time_unit_multipliers = {
    'seconds': 1,
    'minutes': 60,
    'hours': 3600,
    'days': 86400,
    'weeks': 604800,
    'months': 2592000,
    'years': 31536000
}
df['duration_unit'].fillna(1, inplace=True)  # fill missing values with 1 (seconds)

# Calculate calculated_duration_value
df['total_seconds'] = df['duration_value'] * df['duration_unit'].map(time_unit_multipliers.get)

df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comments_shapes,calculated_duration,duration_value,duration_unit,total_seconds
0,10/10/1949 20:30,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,round,45 minutes,45.0,minutes,2700.0
1,10/10/1949 21:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,cross,2 hrs,2.0,hours,7200.0
2,10/10/1955 17:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,,20 seconds,20.0,seconds,20.0
3,10/10/1956 21:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,other,2 hour,2.0,hours,7200.0
4,10/10/1960 20:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,,15 minutes,15.0,minutes,900.0


In [24]:
# Convert datetime column to datetime object
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce') # coerce errors to NaT
# drop rows with NaT values
df.dropna(subset=['datetime'], inplace=True)

# # Create new columns for year, month, day, hour, minute, and second
# df['year'] = df['datetime'].dt.year
# df['month'] = df['datetime'].dt.month
# df['day'] = df['datetime'].dt.day
# df['hour'] = df['datetime'].dt.hour
# df['minute'] = df['datetime'].dt.minute
# df['second'] = df['datetime'].dt.second
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79615 entries, 0 to 80331
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   datetime              79615 non-null  datetime64[ns]
 1   city                  79615 non-null  object        
 2   shape                 77739 non-null  object        
 3   duration (seconds)    79615 non-null  object        
 4   duration (hours/min)  79615 non-null  object        
 5   comments              79615 non-null  object        
 6   date posted           79615 non-null  object        
 7   latitude              79615 non-null  object        
 8   longitude             79615 non-null  float64       
 9   comments_shapes       79615 non-null  object        
 10  calculated_duration   79615 non-null  object        
 11  duration_value        76071 non-null  float64       
 12  duration_unit         79615 non-null  object        
 13  total_seconds   

In [25]:
# from datetime import datetime

# df['date_posted'] = pd.to_datetime(df['date posted'])
# df['time_since_event'] = df['date_posted'] - df['datetime']
# df['comment_length_words'] = df['comments'].apply(lambda x: len(x.split()))
# df['comment_length_characters'] = df['comments'].apply(lambda x: len(x))
# df['comment_unique_words'] = df['comments'].apply(lambda x: len(set(x.split())))
# df.info()

Consider removing or imputing any remaining null values in your dataset. This will depend on the amount of null values and the importance of the columns they are in.

Check for any outliers or anomalies in your data. Outliers can significantly impact the performance of your model, so it's important to address them if they exist.

Consider transforming any skewed columns using techniques such as log transformation to improve the performance of your model.

If you haven't already, split your data into training and testing sets. This will allow you to evaluate the performance of your model on unseen data.

Choose an appropriate evaluation metric to measure the performance of your model. This will depend on the type of problem you are solving and the goal of your model.

Finally, choose a machine learning algorithm and train your model on the training data. Be sure to fine-tune the hyperparameters of your model to improve its performance.

In [27]:
def clean_text(text):
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # remove numbers
    text = re.sub(r'\d+', '', text)
    # convert text to lowercase
    text = text.lower()
    return text

# Create a new column called `comments_clean` that contains the cleaned comments
df['comments_clean'] = df['comments'].apply(clean_text)
# save a cleaned version of the dataset to project_folder/data/processed
df.to_csv('../data/intermediary/ufos_clean.csv', index=False)
df.head()

,datetime,city,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude,comments_shapes,calculated_duration,duration_value,duration_unit,total_seconds,comments_clean
0,1949-10-10 20:30:00,san marcos,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111,round,45 minutes,45.0,minutes,2700.0,this event took place in early fall around it...
1,1949-10-10 21:00:00,lackland afb,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082,cross,2 hrs,2.0,hours,7200.0,lackland afb tx lights racing across the sky...
2,1955-10-10 17:00:00,chester (uk/england),circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667,,20 seconds,20.0,seconds,20.0,greenorange circular disc over chester england
3,1956-10-10 21:00:00,edna,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833,other,2 hour,2.0,hours,7200.0,my older brother and twin sister were leaving ...
4,1960-10-10 20:00:00,kaneohe,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611,,15 minutes,15.0,minutes,900.0,as a marine st lt flying an fjb fighterattack ...
